此專案(face_project)環境
1.虛擬環境
2.使用python3.9,請確認右上角為python39

In [ ]:
import sys
# sys.path.append('C:\\Users\\Leo\\Anaconda3\\envs\\face_project\\Lib\\site-packages')
print("path -->",sys.path,"\n")
print("V -->",sys.version,"\n")

#確認是否使用虛擬環境內的python版本，如果不是須設定
print("exe -->",sys.executable,"\n")


In [ ]:
#-----------------------------------收集照片用
import os
import time
import uuid
import cv2

#必須先有pic資料夾以及who資料夾(who改成人員名稱)
IMAGES_PATH = os.path.join('pic','Apple')
#30，為拍照張數
number_images = 30
#開鏡頭
cap = cv2.VideoCapture(0)
#抓取照片，並存入資料夾底下./pic/who

for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    # 當按下Esc結束迴圈
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break

#關閉相機
cap.release()
cv2.destroyAllWindows()

In [ ]:
#------------------------------即時辨識打卡系統(openccv + arcface)
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#opencv人臉辨識檔案
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#開啟相機
cap = cv2.VideoCapture(0)

#定義顯示文字參數
text = ''
name_org = (40, 80)
time_org = (40, 90)
fontFace = cv2.FONT_HERSHEY_COMPLEX
fontScale = 1
fontcolor = (0, 255, 0) # BGR
thickness = 2 
lineType = 4
bottomLeftOrigin = 1

#開始辨識
while True:
    #讀取影像
    _, img_raw = cap.read()
    #轉成灰階模式
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #此方法的任務是檢測不同大小的對象，並返回矩形的列表
    #詳細解釋可參考 https://blog.csdn.net/leaf_zizi/article/details/107637433
    faces = face_cascade.detectMultiScale(img_raw, 1.1, 8)
    # print("faces-->",faces)

    for i in range(len(faces)):
        try:
            #臉部位置
            face_x, face_y, face_w, face_h = faces[i]
            
            #抓出人臉
            img = img_raw[int(face_y):int(face_y+face_h), int(face_x):int(face_x+face_w)]

            #標記人臉範圍，名字位置，時間位置
            # for (x, y, w, h) in faces:
            cv2.rectangle(img_raw, (face_x, face_y), (face_x+face_w, face_y+face_h), (0, 255, 0), 2)
            name_org = (face_x, face_y-30)
            time_org = (face_x, face_y)
                
            #透過arcface辨識，img為辨識目標位置，db_path為影像資料庫位置
            df = DeepFace.find(img, db_path = "pic", model_name = 'ArcFace',enforce_detection=False)
            print(df.head())

            #驗證圖片
            # result = DeepFace.verify(img, "pic", model_name = 'ArcFace',enforce_detection=False)
            # print(result)
            
            try:
                #抓取辨識名字
                name = df.loc[0].values[0].split('/')[-2].split('\\')[-1]
                #抓取當前時間
                Time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

                #顯示名字、時間到畫面上
                cv2.putText(img_raw, name, name_org, fontFace, fontScale, fontcolor, thickness, lineType)
                cv2.putText(img_raw, Time, time_org, fontFace, fontScale, fontcolor, thickness, lineType)

                #打卡紀錄
                punch_path = 'output.txt'
                with open(punch_path, 'a') as f:
                    f.write(f"{name} at {Time} punch cad.\n")

                print(f"我辨識這位是 {name}。")
            except:
                pass
        except:
            pass
    #正常視窗大小
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    #秀出圖片                                           
    cv2.imshow('img', img_raw)                     

    # 當按下Esc結束迴圈
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
#關閉相機
cap.release()
cv2.destroyAllWindows()



In [ ]:
#--------------------------------------------------輸入照片圖片模式(retinaface + arcface)
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#圖像辨識-------------------------------------------------------

#讀取圖像
img = cv2.imread("test_img\\0002.jpg")


#轉正臉
# faces2 = RetinaFace.extract_faces(img, align = True)
# for face in faces2:
#     plt.imshow(face)
#     plt.show()

#透過RetinaFace辨識人像，默認閾值設置為 0.9。如果要以低解析度檢測人臉，則可以降低它。
faces  = RetinaFace.detect_faces(img,threshold = 0.9)

#圖像參數
faces.keys()

try:
    #取出參數
    for key in faces.keys():

        #第X張人像總參數
        identity = faces[key]

        #人像矩形邊界參數
        facial_area = identity["facial_area"]

        #特徵標記
        landmarks = identity["landmarks"]

        #將numpy.float數值轉換為int，因為cv2.circle參數center格式需要int
        le = tuple(map(int,landmarks["left_eye"]))
        re = tuple(map(int,landmarks["right_eye"]))
        n  = tuple(map(int,landmarks["nose"]))
        lm = tuple(map(int,landmarks["mouth_left"]))
        rm = tuple(map(int,landmarks["mouth_right"]))
        
        #框出人像
        cv2.rectangle(img, (facial_area[2],facial_area[3]),(facial_area[0],facial_area[1]),(255,255,255),1)
        #標記左眼位置
        cv2.circle(img, le, 1, (0, 0, 255), -1)
        #標記右眼位置
        cv2.circle(img, re, 1, (0, 0, 255), -1)
        #標記鼻子位置
        cv2.circle(img, n, 1, (0, 0, 255), -1)
        #標記左嘴角位置
        cv2.circle(img, lm, 1, (0, 0, 255), -1)
        #標記右嘴角位置
        cv2.circle(img, rm, 1, (0, 0, 255), -1)   

    #設定顯示圖片大小
    plt.figure(figsize = (5,5))
    #使圖片顯示完整
    plt.imshow(img[:,:,::-1])
    #顯示圖片
    plt.show()

#-------find & verfiy distance太大的時候，建議可以縮小threshold的大小-----------
#-------修改threshold檔案為\deepface\commons\distance.py----------------------

    #透過arcface辨識，img為辨識目標位置，db_path為影像資料庫位置
    #find()預設有轉正臉
    df = DeepFace.find(img, db_path = "pic\\Ester", model_name = 'ArcFace',detector_backend = 'retinaface',enforce_detection=False)
    #顯示比對照片路徑&consine值
    print(df.head())

    #驗證圖片
    result = DeepFace.verify(img, "test_img\\0009.jpg", model_name = 'ArcFace',detector_backend = 'retinaface',enforce_detection=False)
    print(result)
#----------------------------------------------------------------------------

    #讀取圖像
    img2 = cv2.imread("test_img\\0009.jpg")
    #設定顯示圖片大小
    plt.figure(figsize = (5,5))
    #使圖片顯示完整
    plt.imshow(img2[:,:,::-1])
    #顯示圖片
    plt.show()


    try:
        #抓取辨識名字
        name = df.loc[0].values[0].split('/')[-2].split('\\')[-1]
        #抓取當前時間
        Time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        # cv2.putText(img, name, name_org, fontFace, fontScale, fontcolor, thickness, lineType)
        # cv2.putText(img, Time, time_org, fontFace, fontScale, fontcolor, thickness, lineType)
        print(f"我辨識這位是 {name}。")
    except:
        pass
except:
    print("!! 辨識失敗 !!")    


In [ ]:
#--------------------------------------------------輸入影片模式(retinaface + arcface)
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#影像辨識-------------------------------------------------------
cap = cv2.VideoCapture('test.mp4')

while(True):
    _, img = cap.read()
    try:

    #圖像辨識-------------------------------------------------------

    #透過RetinaFace辨識人像，默認閾值設置為 0.9。如果要以低解析度檢測人臉，則可以降低它。
        faces  = RetinaFace.detect_faces(img,threshold = 0.9)

        #圖像參數
        faces.keys()
        
        #取出參數
        for key in faces.keys():

            #第X張人像總參數
            identity = faces[key]

            #人像矩形邊界參數
            facial_area = identity["facial_area"]

            #特徵標記
            landmarks = identity["landmarks"]

            #將numpy.float數值轉換為int，因為cv2.circle參數center格式需要int
            le = tuple(map(int,landmarks["left_eye"]))
            re = tuple(map(int,landmarks["right_eye"]))
            n  = tuple(map(int,landmarks["nose"]))
            lm = tuple(map(int,landmarks["mouth_left"]))
            rm = tuple(map(int,landmarks["mouth_right"]))
            
            #框出人像
            cv2.rectangle(img, (facial_area[2],facial_area[3]),(facial_area[0],facial_area[1]),(255,255,255),1)
            #標記左眼位置
            cv2.circle(img, le, 1, (0, 0, 255), -1)
            #標記右眼位置
            cv2.circle(img, re, 1, (0, 0, 255), -1)
            #標記鼻子位置
            cv2.circle(img, n, 1, (0, 0, 255), -1)
            #標記左嘴角位置
            cv2.circle(img, lm, 1, (0, 0, 255), -1)
            #標記右嘴角位置
            cv2.circle(img, rm, 1, (0, 0, 255), -1)   

        # #設定顯示圖片大小
        # plt.figure(figsize = (5,5))
        # #使圖片顯示完整
        # plt.imshow(img[:,:,::-1])
        # #顯示圖片
        # plt.show()

        #透過arcface辨識，img為辨識目標位置，db_path為影像資料庫位置
        df = DeepFace.find(img, db_path = "pic\\Ester", model_name = 'ArcFace',detector_backend = 'retinaface',enforce_detection=False)
        print(df.head())

        #驗證圖片
        result = DeepFace.verify(img, "pic\\Ester\\ester2000.jpg", model_name = 'ArcFace',detector_backend = 'retinaface',enforce_detection=False)
        print(result)
        
        try:
            #抓取辨識名字
            name = df.loc[0].values[0].split('/')[-2].split('\\')[-1]
            #抓取當前時間
            Time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            # cv2.putText(img, name, name_org, fontFace, fontScale, fontcolor, thickness, lineType)
            # cv2.putText(img, Time, time_org, fontFace, fontScale, fontcolor, thickness, lineType)
            print(f"我辨識這位是 {name}。")
        except:
            pass
    except:
        print("!! 辨識失敗 !!")
            
    # 當按下Esc結束迴圈
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
    cv2.imshow("video",img)
cap.release()
cv2.destroyAllWindows()

以下為人臉辨識4大步驟拆解

In [ ]:
#----抓臉
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#讀取圖像
img = cv2.imread("test_img\\0009.jpg")
#透過RetinaFace辨識人像，默認閾值設置為 0.9。如果要以低解析度檢測人臉，則可以降低它。
faces  = RetinaFace.detect_faces(img,threshold = 0.9)

#圖像參數
#'score'越高此圖越像人臉， 'facial_area'座標參數，'landmarks': 包含眼睛鼻子嘴角的座標
print("faces --> ",faces,"\n")
# print("faces.keys() --> ",faces.keys(),"\n")

#取出參數
for key in faces.keys():

    #第X張人像總參數
    identity = faces[key]

    #人像矩形邊界參數
    facial_area = identity["facial_area"]

    #特徵標記
    landmarks = identity["landmarks"]

    #將numpy.float數值轉換為int，因為cv2.circle參數center格式需要int
    le = tuple(map(int,landmarks["left_eye"]))
    re = tuple(map(int,landmarks["right_eye"]))
    n  = tuple(map(int,landmarks["nose"]))
    lm = tuple(map(int,landmarks["mouth_left"]))
    rm = tuple(map(int,landmarks["mouth_right"]))
    
    #框出人像
    cv2.rectangle(img, (facial_area[2],facial_area[3]),(facial_area[0],facial_area[1]),(255,255,255),1)
    #標記左眼位置
    cv2.circle(img, le, 1, (0, 0, 255), -1)
    #標記右眼位置
    cv2.circle(img, re, 1, (0, 0, 255), -1)
    #標記鼻子位置
    cv2.circle(img, n, 1, (0, 0, 255), -1)
    #標記左嘴角位置
    cv2.circle(img, lm, 1, (0, 0, 255), -1)
    #標記右嘴角位置
    cv2.circle(img, rm, 1, (0, 0, 255), -1)   
#設定顯示圖片大小
plt.figure(figsize = (5,5))
#使圖片顯示完整
plt.imshow(img[:,:,::-1])
#顯示圖片
plt.show()


In [ ]:
#----人臉轉正
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#讀取圖像
img = cv2.imread("test_img\\0009.jpg")

#轉正臉
#透過雙眼以及鼻子的位置，調整出正臉
faces2 = RetinaFace.extract_faces(img, align = True)

for face in faces2:
    plt.imshow(face)
    plt.show()

# plt.imsave('img2.png',image)
# plt.savefig("img2.jpg")

In [ ]:
#----特徵向量
print(faces)

In [ ]:
#----比對
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time

#讀取圖像
img = cv2.imread("test_img\\0009.jpg")
img2 = cv2.imread("test_img\\0002.jpg")


#驗證圖片
result = DeepFace.verify(img, img2, model_name = 'ArcFace',detector_backend = 'retinaface',enforce_detection=False)
print("verify --> ",result,'\n')

#透過RetinaFace辨識人像，默認閾值設置為 0.9。如果要以低解析度檢測人臉，則可以降低它。
faces  = RetinaFace.detect_faces(img,threshold = 0.9)
faces2  = RetinaFace.detect_faces(img2,threshold = 0.9)

#圖像參數
print("faces --> ",faces,"\n")
# print("faces.keys() --> ",faces.keys(),"\n")

print("faces2 --> ",faces2,"\n")
# print("faces2.keys() --> ",faces2.keys(),"\n")

#取出參數
for twofaces in [faces,faces2]:
    for key in twofaces.keys():

        #第X張人像總參數
        identity = faces[key]

        #人像矩形邊界參數
        facial_area = identity["facial_area"]

        #特徵標記
        landmarks = identity["landmarks"]

        #將numpy.float數值轉換為int，因為cv2.circle參數center格式需要int
        le = tuple(map(int,landmarks["left_eye"]))
        re = tuple(map(int,landmarks["right_eye"]))
        n  = tuple(map(int,landmarks["nose"]))
        lm = tuple(map(int,landmarks["mouth_left"]))
        rm = tuple(map(int,landmarks["mouth_right"]))
        
        if twofaces == faces:
            img = img
        else:
            img = img2
        #框出人像
        cv2.rectangle(img, (facial_area[2],facial_area[3]),(facial_area[0],facial_area[1]),(255,255,255),1)
        #標記左眼位置
        cv2.circle(img, le, 1, (0, 0, 255), -1)
        #標記右眼位置
        cv2.circle(img, re, 1, (0, 0, 255), -1)
        #標記鼻子位置
        cv2.circle(img, n, 1, (0, 0, 255), -1)
        #標記左嘴角位置
        cv2.circle(img, lm, 1, (0, 0, 255), -1)
        #標記右嘴角位置
        cv2.circle(img, rm, 1, (0, 0, 255), -1)   

        #設定顯示圖片大小
        plt.figure(figsize = (5,5))
        #使圖片顯示完整
        plt.imshow(img[:,:,::-1])
        #顯示圖片
        plt.show()



In [ ]:
#pkl檔案內容
import pickle
path = 'pic\\representations_arcface.pkl'

f = open(path,'rb')
data = pickle.load(f)

print(data[0])
print(len(data[0][1]))